<a href="https://colab.research.google.com/github/claudiakitova/llm_coder/blob/main/finetuning_baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install git+https://github.com/huggingface/transformers.git@main bitsandbytes  # we need latest transformers for this
!pip install git+https://github.com/huggingface/peft.git@4c611f4
!pip install datasets==2.10.1
import locale # colab workaround
locale.getpreferredencoding = lambda: "UTF-8" # colab workaround
!pip install wandb
!pip install scipy
!pip install huggingface_hub
!pip install tokenization
!pip install pyspark

In [ ]:
# убить ядро, если необходимо: colab тупит с зависимостями, и иногда проще всего перезапуститься после загрузки первой ячейки с зависимостями

# import os
# os.kill(os.getpid(), 9)

In [ ]:
from pyspark.sql.functions import pandas_udf, PandasUDFType
import tokenization
from pyspark.sql import SparkSession
import pandas as pd
import io
from google.colab import files
from datetime import datetime
import sys
import torch
import json
import random
import datasets
from datasets import Dataset
from peft import PeftModel

from peft import (
    LoraConfig,
    get_peft_model,
    get_peft_model_state_dict,
    prepare_model_for_int8_training,
    set_peft_model_state_dict,
)
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForSeq2Seq

In [ ]:
# загружаем и форматируем датасет
uploaded = files.upload()
df = pd.read_excel(io.BytesIO(uploaded['data.xlsx']))
df.to_json(r'data.json', orient='records', force_ascii=False)
with open("data.json", "r") as f:
    data = json.load(f)

dataset_dict = {"question": [item["question"] for item in data], "context": [item["context"] for item in data], "answer": [item["answer"] for item in data]}

data = Dataset.from_dict(dataset_dict)
data

Dataset({
    features: ['question', 'context', 'answer'],
    num_rows: 40
})

In [ ]:
# проверяем, что в датасете
print(data[5])

{'question': 'Покажи всех контрагентов из справочника Контрагенты, выведи все поля таблицы, кроме табличных частей\n\n\n', 'context': 'Покажи всех контрагентов из справочника Контрагенты, выведи все поля таблицы, кроме табличных частей\nYour request must be executed in my 1C database. Use the database structure below. Use in your answer only the names of tables and fields from this structure. Tables can only be linked by fields of the same type.\n# 1C database structure:\n#### Справочник.СтраныМира\nИмяПредопределенныхДанных (вид - реквизит; тип - Строка)\nПредопределенный (вид - реквизит; тип - Булево)\nСсылка (вид - реквизит; тип - Справочник.СтраныМира)\nПометкаУдаления (вид - реквизит; тип - Булево)\nНаименование (вид - реквизит; тип - Строка)\nКод (вид - реквизит; тип - Строка)\nНаименованиеПолное (вид - реквизит; тип - Строка)\nКодАльфа2 (вид - реквизит; тип - Строка)\nКодАльфа3 (вид - реквизит; тип - Строка)\nУчастникЕАЭС (вид - реквизит; тип - Булево)\nМеждународноеНаименование

In [ ]:
# удостоверяемся, что формат верный
type(data)

datasets.arrow_dataset.Dataset

In [ ]:
# делим датасет на тренировочную и тестовую выборки

train_dataset = data.train_test_split(test_size=0.15)["train"]
eval_dataset = data.train_test_split(test_size=0.15)["test"]

print(train_dataset)
print(eval_dataset)

Dataset({
    features: ['question', 'context', 'answer'],
    num_rows: 34
})
Dataset({
    features: ['question', 'context', 'answer'],
    num_rows: 6
})


In [ ]:
# системный промпт
system_message = """
You are a professional programmer of the 1C:Enterprise system.
Your goal is to provide the user with a correct and elegant query to the 1C database in the 1C query language that will solve his problem.
"""

### Базовые модели

In [ ]:
%%time
# задаем список моделей, которые будем использовать
base_model_list = [
    "codellama/CodeLlama-7b-hf",            # CodLLama
    "bigcode/starcoderbase",                # StarCoderBase
    "deepseek-ai/deepseek-coder-6.7b-base", # DeepSeekCoder
    "chatdb/natural-sql-7b",                # Natural-SQL (на базе DeepSeekCoder)
    "Phind/Phind-CodeLlama-34B-v2",         # Phind CodLLama (на базе CodLLama)
    # список можно пополнять или закомментировать неиспользуемые модели
]

base_tokenizer_list = [
    "codellama/CodeLlama-7b-hf",            # CodLLama
    "bigcode/starcoderbase",                # StarCoderBase
    "deepseek-ai/deepseek-coder-6.7b-base", # DeepSeekCoder
    "chatdb/natural-sql-7b",                # Natural-SQL (на базе DeepSeekCoder)
    "Phind/Phind-CodeLlama-34B-v2",         # Phind CodLLama (на базе CodLLama)
    # список можно пополнять или закомментировать неиспользуемые токенизаторы
]

model_names = base_model_list.copy()

# инициализируем пустые списки для хранения моделей и токенизаторов
loaded_models = []
loaded_tokenizers = []

# загружаем модели
for model_name in base_model_list:
  model = AutoModelForCausalLM.from_pretrained(
      model_name,
      torch_dtype=torch.float16,
      device_map="auto",
  )
  loaded_models.append(model)

# загружаем соответствующие моделям токенизаторы
for tokenizer_name in base_tokenizer_list:
  tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
  loaded_tokenizers.append(tokenizer)

config.json:   0%|          | 0.00/637 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

tokenizer_config.json:   0%|          | 0.00/749 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

CPU times: user 38.9 s, sys: 32.9 s, total: 1min 11s
Wall time: 2min 38s


In [ ]:
print(loaded_tokenizers)

CodeLlamaTokenizerFast(name_or_path='codellama/CodeLlama-7b-hf', vocab_size=32016, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'additional_special_tokens': ['▁<PRE>', '▁<MID>', '▁<SUF>', '▁<EOT>']}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	32007: AddedToken("▁<PRE>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	32008: AddedToken("▁<SUF>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	32009: AddedToken("▁<MID>", rstrip=False, lstrip=False, single_word

In [ ]:
print(loaded_models)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32016, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  (lm_head): Lin

### Токенизация
Применим токенизацию (левый паддинг, потому что он [тратит меньше памяти](https://ai.stackexchange.com/questions/41485/while-fine-tuning-a-decoder-only-llm-like-llama-on-chat-dataset-what-kind-of-pa))

In [ ]:
tokenizer.add_eos_token = True
tokenizer.pad_token_id = 0
tokenizer.padding_side = "left"

Настроим функцию tokenize так, чтобы labels и input_ids были одинаковыми (принцип [self-supervised fine-tuning](https://neptune.ai/blog/self-supervised-learning))

In [ ]:
def tokenize(prompt):
    result = tokenizer(
        prompt,
        truncation=True,
        max_length=512,
        padding=False,
        return_tensors=None,
    )

    # "self-supervised learning" значит, что labels являются одновременно input_ids
    result["labels"] = result["input_ids"].copy()

    return result

Запустим конвертацию каждой точки данных в промпт

In [ ]:
def generate_and_tokenize_prompt(data_point):
  full_prompt = f"""You are a professional programmer of the 1C:Enterprise system.
  Your goal is to provide the user with a correct and elegant query to the 1C database in the 1C query language that will solve his problem.
  ### Input:
  {data_point["question"]}
  ### Context:
  {data_point["context"]}
  ### Response:
  {data_point["answer"]}
  """
  return tokenize(full_prompt)

In [ ]:
tokenized_train_dataset = train_dataset.map(generate_and_tokenize_prompt)
tokenized_val_dataset = eval_dataset.map(generate_and_tokenize_prompt)

Map:   0%|          | 0/34 [00:00<?, ? examples/s]

Map:   0%|          | 0/6 [00:00<?, ? examples/s]

### Применяем LoRA ([Low Rank Adaptation](https://medium.com/@manyi.yim/more-about-loraconfig-from-peft-581cf54643db))

In [ ]:
for model, model_name in zip(loaded_models, model_names):
    model.train()
    model = prepare_model_for_int8_training(model)

    config = LoraConfig(
        r=16,
        lora_alpha=16,
        target_modules=[
            "q_proj",
            "k_proj",
            "v_proj",
            "o_proj",
        ],
        lora_dropout=0.05,
        bias="none",
        task_type="CAUSAL_LM",
    )
    model = get_peft_model(model, config)

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:133: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


Weights & Biases (опционально)

In [ ]:
wandb_project = "sql-try2-coder"
if len(wandb_project) > 0:
    os.environ["WANDB_PROJECT"] = wandb_project

### Обучение

In [ ]:
# проверка GPU
!nvidia-smi

Mon Mar  4 21:29:27 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla V100-SXM2-16GB           Off | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P0              43W / 300W |  13682MiB / 16384MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
if torch.cuda.device_count() > 1:
    model.is_parallelizable = True
    model.model_parallel = True

In [ ]:
%%time
# если закончилась память GPU - меняем размер per_device_train_batch_size
# переменная gradient_accumulation_steps должна гарантировать, что это не повлияет на обучение

batch_size = 128
per_device_train_batch_size = 16
gradient_accumulation_steps = batch_size // per_device_train_batch_size
output_dir = f"1c-{model_name}"

training_args = TrainingArguments(
        per_device_train_batch_size=per_device_train_batch_size,
        gradient_accumulation_steps=gradient_accumulation_steps,
        warmup_steps=100,
        max_steps=400,
        learning_rate=3e-4,
        fp16=True,
        logging_steps=10,
        optim="adamw_torch",
        evaluation_strategy="steps", # if val_set_size > 0 else "no",
        save_strategy="steps",
        eval_steps=20,
        save_steps=20,
        output_dir=output_dir,
        # save_total_limit=3,
        load_best_model_at_end=False,
        # ddp_find_unused_parameters=False if ddp else None,
        group_by_length=True, # группируем последовательности примерно одинаковой длины для ускорения обучения
        report_to="wandb", # if use_wandb else "none",
        run_name=f"{model_name}-{datetime.now().strftime('%Y-%m-%d-%H-%M')}", # if use_wandb else None,
    )

CPU times: user 851 µs, sys: 0 ns, total: 851 µs
Wall time: 864 µs


In [ ]:
for model, model_name in zip(loaded_models, model_names):
  trainer = Trainer(
      model=model,
     train_dataset=tokenized_train_dataset,
     eval_dataset=tokenized_val_dataset,
     args=training_args,
     data_collator=DataCollatorForSeq2Seq(
         tokenizer, pad_to_multiple_of=8, return_tensors="pt", padding=True
     ),
 )

In [ ]:
for model, model_name in zip(loaded_models, model_names):
  model.config.use_cache = False

  old_state_dict = model.state_dict
  model.state_dict = (lambda self, *_, **__: get_peft_model_state_dict(self, old_state_dict())).__get__(
      model, type(model)
  )
  if torch.__version__ >= "2" and sys.platform != "win32":
      print("Compiling the model")
      model = torch.compile(model)

compiling the model


In [ ]:
trainer.train()

### Чекпойнт

In [ ]:
# проверяем на CodeLlama для экономии памяти; вне режима экономии нужно пройтись по списку моделей из loaded_models
base_model = "codellama/CodeLlama-7b-hf"
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    load_in_8bit=True,
    torch_dtype=torch.float16,
    device_map="auto",
)
tokenizer = AutoTokenizer.from_pretrained("codellama/CodeLlama-7b-hf")

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
model = PeftModel.from_pretrained(model, output_dir)

In [ ]:
eval_prompt = """You are a professional programmer of the 1C:Enterprise system.
Your goal is to provide the user with a correct and elegant query to the 1C database in the 1C query language that will solve his problem.

### Input:
Выбери все элементы из справочника номенклатура. Покажи поля Ссылка, Код и ПолноеНаименование

### Context:
Выбери все элементы из справочника номенклатура. Покажи поля Ссылка, Код и ПолноеНаименование
Your request must be executed in my 1C database. Use the database structure below. Use in your answer only the names of tables and fields from this structure. Tables can only be linked by fields of the same type.
# 1C database structure:
#### Справочник.Номенклатура
ИмяПредопределенныхДанных (вид - реквизит; тип - Строка)
Предопределенный (вид - реквизит; тип - Булево)
Ссылка (вид - реквизит; тип - Справочник.Номенклатура)
ПометкаУдаления (вид - реквизит; тип - Булево)
ЭтоГруппа (вид - реквизит; тип - Булево)
Родитель (вид - реквизит; тип - Справочник.Номенклатура)
Наименование (вид - реквизит; тип - Строка)
Код (вид - реквизит; тип - Строка)
ЕдиницаИзмерения (вид - реквизит; тип - Справочник.УпаковкиЕдиницыИзмерения)
Артикул (вид - реквизит; тип - Строка)
ВариантОформленияПродажи (вид - реквизит; тип - Перечисление.ВариантыОформленияПродажи)
ВесЕдиницаИзмерения (вид - реквизит; тип - Справочник.УпаковкиЕдиницыИзмерения)
ВесЗнаменатель (вид - реквизит; тип - Число)
ВесИспользовать (вид - реквизит; тип - Булево)
ВесМожноУказыватьВДокументах (вид - реквизит; тип - Булево)
ВесЧислитель (вид - реквизит; тип - Число)
ВестиУчетПоГТД (вид - реквизит; тип - Булево)
ВестиУчетСертификатовНоменклатуры (вид - реквизит; тип - Булево)
ВидАлкогольнойПродукции (вид - реквизит; тип - Справочник.ВидыАлкогольнойПродукции)
ВидНоменклатуры (вид - реквизит; тип - Справочник.ВидыНоменклатуры)
ГруппаДоступа (вид - реквизит; тип - Справочник.ГруппыДоступаНоменклатуры)
АлкогольнаяПродукция (вид - реквизит; тип - Булево)
НаименованиеПолное (вид - реквизит; тип - Строка)
ЕдиницаИзмеренияСрокаГодности (вид - реквизит; тип - Перечисление.ЕдиницыИзмеренияВремени)
ЕстьТоварыДругогоКачества (вид - реквизит; тип - Булево)
ИмпортнаяАлкогольнаяПродукция (вид - реквизит; тип - Булево)
ДлинаЕдиницаИзмерения (вид - реквизит; тип - Справочник.УпаковкиЕдиницыИзмерения)
ДлинаЗнаменатель (вид - реквизит; тип - Число)
ДлинаИспользовать (вид - реквизит; тип - Булево)
ДлинаМожноУказыватьВДокументах (вид - реквизит; тип - Булево)
ДлинаЧислитель (вид - реквизит; тип - Число)
ИспользованиеХарактеристик (вид - реквизит; тип - Перечисление.ВариантыИспользованияХарактеристикНоменклатуры)
ИспользоватьИндивидуальныйШаблонЦенника (вид - реквизит; тип - Булево)
ИспользоватьИндивидуальныйШаблонЭтикетки (вид - реквизит; тип - Булево)
ИспользоватьУпаковки (вид - реквизит; тип - Булево)
Качество (вид - реквизит; тип - Перечисление.ГрадацииКачества)
КодДляПоиска (вид - реквизит; тип - Строка)
Марка (вид - реквизит; тип - Справочник.Марки)
НаборУпаковок (вид - реквизит; тип - Справочник.НаборыУпаковок)
ГруппаФинансовогоУчета (вид - реквизит; тип - Справочник.ГруппыФинансовогоУчетаНоменклатуры)
НоменклатураМногооборотнаяТара (вид - реквизит; тип - Справочник.Номенклатура)
ОбъемДАЛ (вид - реквизит; тип - Число)
Описание (вид - реквизит; тип - Строка)
ПодакцизныйТовар (вид - реквизит; тип - Булево)
ПоставляетсяВМногооборотнойТаре (вид - реквизит; тип - Булево)
Производитель (вид - реквизит; тип - Справочник.Производители)
ПроизводительИмпортерКонтрагент (вид - реквизит; тип - Справочник.Контрагенты)
СкладскаяГруппа (вид - реквизит; тип - Справочник.СкладскиеГруппыНоменклатуры)
СрокГодности (вид - реквизит; тип - Число)
СтавкаНДС (вид - реквизит; тип - Справочник.СтавкиНДС)
ТипНоменклатуры (вид - реквизит; тип - Перечисление.ТипыНоменклатуры)
ТоварнаяКатегория (вид - реквизит; тип - Справочник.ТоварныеКатегории)
ФайлКартинки (вид - реквизит; тип - Справочник.НоменклатураПрисоединенныеФайлы)
ФайлОписанияДляСайта (вид - реквизит; тип - Справочник.НоменклатураПрисоединенныеФайлы)
ОбъемЕдиницаИзмерения (вид - реквизит; тип - Справочник.УпаковкиЕдиницыИзмерения)
ОбъемЗнаменатель (вид - реквизит; тип - Число)
ОбъемИспользовать (вид - реквизит; тип - Булево)
ОбъемМожноУказыватьВДокументах (вид - реквизит; тип - Булево)
ОбъемЧислитель (вид - реквизит; тип - Число)
ХарактеристикаМногооборотнаяТара (вид - реквизит; тип - Справочник.ХарактеристикиНоменклатуры)
ПлощадьЕдиницаИзмерения (вид - реквизит; тип - Справочник.УпаковкиЕдиницыИзмерения)
ПлощадьЗнаменатель (вид - реквизит; тип - Число)
СхемаОбеспечения (вид - реквизит; тип - Справочник.СхемыОбеспечения)
СпособОбеспеченияПотребностей (вид - реквизит; тип - Справочник.СпособыОбеспеченияПотребностей)
ПлощадьИспользовать (вид - реквизит; тип - Булево)
ПлощадьМожноУказыватьВДокументах (вид - реквизит; тип - Булево)
ПлощадьЧислитель (вид - реквизит; тип - Число)
ЦеноваяГруппа (вид - реквизит; тип - Справочник.ЦеновыеГруппы)
ШаблонЦенника (вид - реквизит; тип - Справочник.ШаблоныЭтикетокИЦенников)
ЕдиницаДляОтчетов (вид - реквизит; тип - Справочник.УпаковкиЕдиницыИзмерения)
КоэффициентЕдиницыДляОтчетов (вид - реквизит; тип - Число)
ШаблонЭтикетки (вид - реквизит; тип - Справочник.ШаблоныЭтикетокИЦенников)
СезоннаяГруппа (вид - реквизит; тип - Справочник.СезонныеГруппы)
КоллекцияНоменклатуры (вид - реквизит; тип - Справочник.КоллекцииНоменклатуры)
Принципал (вид - реквизит; тип - Справочник.ОрганизацииСправочник.Партнеры)
Контрагент (вид - реквизит; тип - Справочник.ОрганизацииСправочник.Контрагенты)
РейтингПродаж (вид - реквизит; тип - Справочник.РейтингиПродажНоменклатуры)
ОбособленнаяЗакупкаПродажа (вид - реквизит; тип - Булево)
ГруппаАналитическогоУчета (вид - реквизит; тип - Справочник.ГруппыАналитическогоУчетаНоменклатуры)
КодТНВЭД (вид - реквизит; тип - Справочник.КлассификаторТНВЭД)
КодОКВЭД (вид - реквизит; тип - Справочник.КлассификаторВидовЭкономическойДеятельности)
КодОКП (вид - реквизит; тип - Справочник.ОбщероссийскийКлассификаторПродукции)
ОблагаетсяНДПИПоПроцентнойСтавке (вид - реквизит; тип - Булево)
ВладелецСерий (вид - реквизит; тип - Справочник.ВидыНоменклатуры)
ВладелецХарактеристик (вид - реквизит; тип - Справочник.ВидыНоменклатуры)
ВладелецТоварныхКатегорий (вид - реквизит; тип - Справочник.ВидыНоменклатуры)
Крепость (вид - реквизит; тип - Число)
ОсобенностьУчета (вид - реквизит; тип - Перечисление.ОсобенностиУчетаНоменклатуры)
ПродукцияМаркируемаяДляГИСМ (вид - реквизит; тип - Булево)
КиЗГИСМ (вид - реквизит; тип - Булево)
КиЗГИСМВид (вид - реквизит; тип - Перечисление.ВидыКиЗГИСМ)
КиЗГИСМСпособВыпускаВОборот (вид - реквизит; тип - Перечисление.СпособыВыпускаВОборотГИСМ)
КиЗГИСМGTIN (вид - реквизит; тип - Строка)
КиЗГИСМРазмер (вид - реквизит; тип - Перечисление.РазмерыКиЗГИСМ)
ПодконтрольнаяПродукцияВЕТИС (вид - реквизит; тип - Булево)
АлкогольнаяПродукцияВоВскрытойТаре (вид - реквизит; тип - Булево)
КодРаздел7ДекларацииНДС (вид - реквизит; тип - Справочник.КодыОперацийРаздела7ДекларацииПоНДС)
ОблагаетсяНДСУПокупателя (вид - реквизит; тип - Булево)
КодОКВЭД2 (вид - реквизит; тип - Справочник.КлассификаторОКВЭД2)
КодОКПД2 (вид - реквизит; тип - Справочник.КлассификаторОКПД2)
КодВидаНоменклатурнойКлассификации (вид - реквизит; тип - Строка)
НаименованиеВидаНоменклатурнойКлассификации (вид - реквизит; тип - Строка)
ЕдиницаИзмеренияТНВЭД (вид - реквизит; тип - Справочник.УпаковкиЕдиницыИзмерения)
СтранаПроисхождения (вид - реквизит; тип - Справочник.СтраныМира)
ПрослеживаемыйТовар (вид - реквизит; тип - Булево)
КодТРУ (вид - реквизит; тип - Строка)

#### Справочник.УпаковкиЕдиницыИзмерения
ИмяПредопределенныхДанных (вид - реквизит; тип - Строка)
Предопределенный (вид - реквизит; тип - Булево)
Ссылка (вид - реквизит; тип - Справочник.УпаковкиЕдиницыИзмерения)
ПометкаУдаления (вид - реквизит; тип - Булево)
Владелец (вид - реквизит; тип - Справочник.НоменклатураСправочник.НаборыУпаковок)
Родитель (вид - реквизит; тип - Справочник.УпаковкиЕдиницыИзмерения)
Наименование (вид - реквизит; тип - Строка)
Код (вид - реквизит; тип - Строка)
Безразмерная (вид - реквизит; тип - Булево)
Вес (вид - реквизит; тип - Число)
ВесЕдиницаИзмерения (вид - реквизит; тип - Справочник.УпаковкиЕдиницыИзмерения)
Высота (вид - реквизит; тип - Число)
ВысотаЕдиницаИзмерения (вид - реквизит; тип - Справочник.УпаковкиЕдиницыИзмерения)
Глубина (вид - реквизит; тип - Число)
ГлубинаЕдиницаИзмерения (вид - реквизит; тип - Справочник.УпаковкиЕдиницыИзмерения)
ЕдиницаИзмерения (вид - реквизит; тип - Справочник.УпаковкиЕдиницыИзмерения)
Числитель (вид - реквизит; тип - Число)
Знаменатель (вид - реквизит; тип - Число)
ЛинейныеРазмерыПредставление (вид - реквизит; тип - Строка)
Объем (вид - реквизит; тип - Число)
ОбъемЕдиницаИзмерения (вид - реквизит; тип - Справочник.УпаковкиЕдиницыИзмерения)
СкладскаяГруппа (вид - реквизит; тип - Справочник.СкладскиеГруппыУпаковок)
Типоразмер (вид - реквизит; тип - Справочник.ТипоразмерыУпаковок)
Ширина (вид - реквизит; тип - Число)
ШиринаЕдиницаИзмерения (вид - реквизит; тип - Справочник.УпаковкиЕдиницыИзмерения)
КоличествоУпаковок (вид - реквизит; тип - Число)
ПоставляетсяВМногооборотнойТаре (вид - реквизит; тип - Булево)
НоменклатураМногооборотнаяТара (вид - реквизит; тип - Справочник.Номенклатура)
ХарактеристикаМногооборотнаяТара (вид - реквизит; тип - Справочник.ХарактеристикиНоменклатуры)
МинимальноеКоличествоУпаковокМногооборотнойТары (вид - реквизит; тип - Число)
ТипИзмеряемойВеличины (вид - реквизит; тип - Перечисление.ТипыИзмеряемыхВеличин)
НаименованиеПолное (вид - реквизит; тип - Строка)
МеждународноеСокращение (вид - реквизит; тип - Строка)
ТипУпаковки (вид - реквизит; тип - Перечисление.ТипыУпаковокНоменклатуры)

### Response:
"""

model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

model.eval()
with torch.no_grad():
  print(tokenizer.decode(model.generate(**model_input, max_new_tokens=100)[0], skip_special_tokens=True))
